### Problem 1 (50 points) 

Vapor-liquid equilibria data are correlated using two adjustable parameters $A_{12}$ and $A_{21}$ per binary
mixture. For low pressures, the equilibrium relation can be formulated as:

$$
\begin{aligned}
p = & x_1\exp\left(A_{12}\left(\frac{A_{21}x_2}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{water}^{sat}\\
& + x_2\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{1,4 dioxane}^{sat}.
\end{aligned}
$$

Here the saturation pressures are given by the Antoine equation

$$
\log_{10}(p^{sat}) = a_1 - \frac{a_2}{T + a_3},
$$

where $T = 20$($^{\circ}{\rm C}$) and $a_{1,2,3}$ for a water - 1,4 dioxane
system is given below.

|             | $a_1$     | $a_2$      | $a_3$     |
|:------------|:--------|:---------|:--------|
| Water       | 8.07131 | 1730.63  | 233.426 |
| 1,4 dioxane | 7.43155 | 1554.679 | 240.337 |


The following table lists the measured data. Recall that in a binary system $x_1 + x_2 = 1$.

|$x_1$ | 0.0 | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
|:-----|:--------|:---------|:--------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
|$p$| 28.1 | 34.4 | 36.7 | 36.9 | 36.8 | 36.7 | 36.5 | 35.4 | 32.9 | 27.7 | 17.5 |

Estimate $A_{12}$ and $A_{21}$ using data from the above table: 

1. Formulate the least square problem; 
2. Since the model is nonlinear, the problem does not have an analytical solution. Therefore, solve it using the gradient descent or Newton's method implemented in HW1; 
3. Compare your optimized model with the data. Does your model fit well with the data?

---








In [6]:
#### PROBLEM 1 Solution 


import torch as t
import numpy as np
from torch.autograd import Variable
import matplotlib.pyplot as plt

# Measured Data for x1 and p 
x1_data = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
p_data = [28.1, 34.4, 36.7, 36.9, 36.8, 36.7, 36.5, 35.4, 32.9, 27.7, 17.5]

# saturation pressure = 10.0**((a[1] - (a[2] / (T + a[3]))))
psat_wat = 10.0**(8.07131-1730.63/(20.0+233.426))
psat_diox = 10.0**(7.43155-1554.679/(20.0+240.337))

#defining a loss func.
def loss(a): 
    loss_value = 0.0
    for i in range(11):
        x1 = x1_data[i]
        p = p_data[i]
        p_fit = x1*psat_wat*t.exp(a[0]*(a[1]*(1-x1)/(a[0]*x1+a[1]*(1-x1)))**2) + (1-x1)*psat_diox*t.exp(a[1]*( a[0]*x1/( a[0]*x1+a[1]*(1-x1) ) )**2)
        loss_value = loss_value + (p_fit-p)**2
    return loss_value
#assume error as 1
error = 1
#initialize A12 and A21 
A = Variable(t.tensor([1.0, 1.0]), requires_grad=True)

# gradient decent
while error >= 0.1:
    loss(A).backward()
    error = t.linalg.norm(A.grad)
    # Perform line search n ensure smaller step size
    s=.1  # intialize step size
    while loss(A-s*A.grad) > loss(A):
        s=.3*s
    with t.no_grad():       
        A -= s*A.grad
        A.grad.zero_()

print(' Optimized value of A12, A21 =', A.data.numpy())
print(' value of loss function =', loss(A).data.numpy())



######################################################



 Optimized value of A12, A21 = [1.9580714 1.6893563]
 value of loss function = 0.6702041


In [2]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
# A simple example of using PyTorch for gradient descent

import torch as t
from torch.autograd import Variable

# Define a variable, make sure requires_grad=True so that PyTorch can take gradient with respect to this variable
x = Variable(t.tensor([1.0, 0.0]), requires_grad=True)

# Define a loss
loss = (x[0] - 1)**2 + (x[1] - 2)**2

# Take gradient
loss.backward()

# Check the gradient. numpy() turns the variable from a PyTorch tensor to a numpy array.
x.grad.numpy()

array([ 0., -4.], dtype=float32)

In [4]:
# Let's examine the gradient at a different x.
x.data = t.tensor([2.0, 1.0])
loss = (x[0] - 1)**2 + (x[1] - 2)**2
loss.backward()
x.grad.numpy()

array([ 2., -6.], dtype=float32)

In [5]:
# Here is a code for gradient descent without line search

import torch as t
from torch.autograd import Variable

x = Variable(t.tensor([1.0, 0.0]), requires_grad=True)

# Fix the step size
a = 0.01

# Start gradient descent
for i in range(1000):  # TODO: change the termination criterion
    loss = (x[0] - 1)**2 + (x[1] - 2)**2
    loss.backward()
    
    # no_grad() specifies that the operations within this context are not part of the computational graph, i.e., we don't need the gradient descent algorithm itself to be differentiable with respect to x
    with t.no_grad():
        x -= a * x.grad
        
        # need to clear the gradient at every step, or otherwise it will accumulate...
        x.grad.zero_()
        
print(x.data.numpy())
print(loss.data.numpy())

[1.        1.9999971]
8.185452e-12
